# Extracting information from an image

> **Universitat de Girona 2023 - Master's thesis**

> **University Master in Data Science**

> **Author:** Llorenç Peirau

## Library and functions

In [ ]:
#%%capture
!pip install -U ExtractTable

In [ ]:
def image_show(image_path):

  import matplotlib.pyplot as plt
  import matplotlib.image as mpimg
  import os

  image_name = os.path.basename(image_path)
  img = mpimg.imread(image_path)
  plt.imshow(img)
  plt.title(image_name)
  return(plt.show())


In [ ]:
def df_names(image_path):

  import pandas as pd
  import os

  image_name = os.path.basename(image_path)
  df_names = ['players_'+image_name[0], 'match_'+image_name[0],
                'goals_'+image_name[0], 'minutesh1_'+image_name[0],
                'minutesh2_'+image_name[0]]

  return(df_names)

In [ ]:
def df_names2(image_path):

  import pandas as pd
  import os

  image_name = os.path.basename(image_path)
  df_names = ['players_'+image_name[0:2], 'match_'+image_name[0:2],
                'goals_'+image_name[0:2], 'minutesh1_'+image_name[0:2],
                'minutesh2_'+image_name[0:2]]

  return(df_names)

In [ ]:
def dict_creation(APIKEY, image_path):

  from ExtractTable import ExtractTable

  et_sess = ExtractTable(api_key=APIKEY)
  print(et_sess.check_usage())
  dict_data = et_sess.process_file(filepath=image_path, output_format="dict")
  return(dict_data)

In [ ]:
def df_creation (image_path, data):

  import pandas as pd
  import os

  image_name = os.path.basename(image_path)

  df_names = ['players_'+image_name[0], 'match_'+image_name[0],
                'goals_'+image_name[0], 'minutesh1_'+image_name[0],
                'minutesh2_'+image_name[0]]

  i = 0
  for name in df_names:
      globals()[name] = pd.DataFrame(data[i])
      globals()[name].columns = globals()[name].iloc[0]
      globals()[name] = globals()[name][1:]
      i += 1

  for i in df_names:
      print(i)


In [ ]:
def df_creation2 (image_path, data):

  import pandas as pd
  import os

  image_name = os.path.basename(image_path)

  df_names = ['players_'+image_name[0:2], 'match_'+image_name[0:2],
                'goals_'+image_name[0:2], 'minutesh1_'+image_name[0:2],
                'minutesh2_'+image_name[0:2]]

  i = 0
  for name in df_names:
      globals()[name] = pd.DataFrame(data[i])
      globals()[name].columns = globals()[name].iloc[0]
      globals()[name] = globals()[name][1:]
      i += 1

  for i in df_names:
      print(i)

**Functions created:**


*   image_show(image_path)
*   df_names(image_path)
*   df_names2(image_path)
*   dict_creation(APIKEY, image_path)
*   df_creation(image_path, data)
*   df_creation2(image_path, data)



## DF Correction functions

### Players Dataset

In [ ]:
def correction_players_dataset(df):

  import pandas as pd

  #Columns name
  players_columns = ["Id_player", "Id_match", "Starting", "Goals", "Free_kick",
                   "Yellow", "Red", "Tackle", "Lost"]
  df.columns = players_columns

  #Replacing characters by numbers
  df.replace({'o': '0', 'O': '0', 'J':'5', 'a':'9'}, regex=True, inplace=True)

  #To numeric format
  df = df.apply(pd.to_numeric, errors='coerce')

  #Save to CSV
  csv_name = df_names_list[0]+".csv"
  df.to_csv(csv_name, index=False)

  return(df)

### Match dataset

In [ ]:
def correction_match_dataset(df):

  import pandas as pd

  #Columns name
  match_columns = ["Id_match", "Id_team", "HoA", "Half", "Shots", "Chances",
                  "Goals", "Corners", "Throw-in", "10m", "Penalty", "Far_post",
                  "Free-kick"]

  df.columns = match_columns

  #Replacing characters by numbers
  df.replace({'o': '0', 'O': '0', 'J':'5'}, regex=True, inplace=True)

  #To numeric format
  df[df.columns.difference(['HoA'])] = df[df.columns.difference(['HoA'])].apply(pd.to_numeric, errors='coerce')

  #Save to CSV
  csv_name = df_names_list[1]+".csv"
  df.to_csv(csv_name, index=False)

  return(df)

### Goals dataset

In [ ]:
def correction_goals_dataset(df):

  import pandas as pd

  #Columns name
  goals_columns = ["Id_match", "Home_goals", "Away_goals", "Id_player", "Half",
                    "Minute", "Play", "Player1", "Player2","Player3","Player4","Player5"]

  df.columns = goals_columns

  df = df.dropna(how='any')

  df.replace({'a': '1', 'to': '5', 'so': '10', 'J':'5', 'M':'11','1 1':'11', 's':'1'},
             regex=True, inplace=True)
  wm_goals = df.loc[:, df.columns != 'Minute']
  wm_goals = wm_goals.astype(int)
  m_goals = df.loc[:, df.columns == 'Minute']
  df = pd.concat([wm_goals,m_goals],axis=1)

  #Save to CSV
  csv_name = df_names_list[2]+".csv"
  df.to_csv(csv_name, index=False)

  return(df)

### Minutesh1 dataset

In [ ]:
def correction_minutesh1_dataset(df):

  import pandas as pd

  #Columns name
  minutesh1_columns = ["Id_match", "Id_player", "Half", "In11", "Out11", "In12",
                  "Out12", "In13", "Out13", "In14", "Out14", "In15",
                  "Out15"]

  df.columns = minutesh1_columns

  df = df.dropna(how='any')

  df.replace({'0 00': '0:00', '0.00': '0:00', '20.00' : '20:00', 'M':'11'},
             regex=True, inplace=True)

  minutes = df.iloc[:, -10:]

  import re
  minutes = minutes.applymap(lambda x: re.sub('[^0-9:]+', '', str(x)))

  minutes = minutes.replace('', '0:00')
  minutes.fillna(0)

  f_minutes = minutes.copy()
  for col in f_minutes.columns:
    f_minutes[col] = pd.to_datetime(f_minutes[col], format='%M:%S')

  col_pairs = [(0, 1), (2, 3), (4,5), (6,7),(8,9)]
  i = 1

  for c1, c2 in col_pairs:
      f_minutes["Round"+str(1)+str(i)] = f_minutes.iloc[:, c1] - f_minutes.iloc[:, c2]
      i+=1

  f_minutes = f_minutes.iloc[:, -5:]
  f_minutes["Total1"] = f_minutes.T.sum()

  for col in f_minutes.columns:
    f_minutes[col] = f_minutes[col].dt.total_seconds().astype(int).apply(lambda x: f'{x // 60}:{x % 60}')

  for col in f_minutes.columns:
    f_minutes[col] = pd.to_datetime(f_minutes[col], format='%M:%S').dt.strftime('%M:%S')

  df = df.iloc[:, :3]

  df = pd.concat([df, minutes,f_minutes], axis=1)

  #Save to CSV
  csv_name = df_names_list[3]+".csv"
  df.to_csv(csv_name, index=False)

  return(df)

### Minutesh2 dataset

In [ ]:
def correction_minutesh2_dataset(df):

  import pandas as pd

  #Columns name
  minutesh2_columns = ["Id_match", "Id_player", "Half", "In21", "Out21", "In22",
                  "Out22", "In23", "Out23", "In24", "Out24", "In25",
                  "Out25"]

  df.columns = minutesh2_columns

  df = df.dropna(how='any')

  df.replace({'0 00': '0:00', '0.00': '0:00', '20.00' : '20:00', 'M':'11'},
             regex=True, inplace=True)
  minutes = df.iloc[:, -10:]

  import re
  minutes = minutes.applymap(lambda x: re.sub('[^0-9:]+', '', str(x)))

  minutes = minutes.replace('', '0:00')
  minutes.fillna(0)

  f_minutes = minutes.copy()
  for col in f_minutes.columns:
    f_minutes[col] = pd.to_datetime(f_minutes[col], format='%M:%S')

  col_pairs = [(0, 1), (2, 3), (4,5), (6,7),(8,9)]
  i = 1

  for c1, c2 in col_pairs:
      f_minutes["Round"+str(2)+str(i)] = f_minutes.iloc[:, c1] - f_minutes.iloc[:, c2]
      i+=1

  f_minutes = f_minutes.iloc[:, -5:]
  f_minutes["Total2"] = f_minutes.T.sum()

  for col in f_minutes.columns:
    f_minutes[col] = f_minutes[col].dt.total_seconds().astype(int).apply(lambda x: f'{x // 60}:{x % 60}')

  for col in f_minutes.columns:
    f_minutes[col] = pd.to_datetime(f_minutes[col], format='%M:%S').dt.strftime('%M:%S')

  df = df.iloc[:, :3]

  df = pd.concat([df, minutes,f_minutes], axis=1)

  #Save to CSV
  csv_name = df_names_list[4]+".csv"
  df.to_csv(csv_name, index=False)

  return(df)

## Example of DF creation

### Match 1

In [ ]:
image_path = #path to image (DRIVE recommended)

image_show(image_path)

In [ ]:
dict_data = dict_creation(API_KEY,image_path)

df_creation (image_path, dict_data)

{'credits': 10, 'queued': 0, 'used': 1}
players_1
match_1
goals_1
minutesh1_1
minutesh2_1


In [ ]:
df_names_list = df_names(image_path)

correction_players_dataset(players_1)

correction_match_dataset(match_1)

correction_goals_dataset(goals_1)

correction_minutesh1_dataset(minutesh1_1)

correction_minutesh2_dataset(minutesh2_1)

,Id_match,Id_player,Half,In21,Out21,In22,Out22,In23,Out23,In24,Out24,In25,Out25,Round21,Round22,Round23,Round24,Round25,Total2
1,1,15,2,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,00:00,00:00,00:00,00:00,00:00,00:00
2,1,7,2,7:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,07:00,00:00,00:00,00:00,00:00,07:00
3,1,5,2,12:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,12:00,00:00,00:00,00:00,00:00,12:00
4,1,4,2,8:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,08:00,00:00,00:00,00:00,00:00,08:00
5,1,10,2,6:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,06:00,00:00,00:00,00:00,00:00,06:00
6,1,8,2,10:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,10:00,00:00,00:00,00:00,00:00,10:00
7,1,2,2,6:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,06:00,00:00,00:00,00:00,00:00,06:00
8,,9,2,18:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,18:00,00:00,00:00,00:00,00:00,18:00
9,1 1,1,2,20:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,20:00,00:00,00:00,00:00,00:00,20:00
10,1,11,2,12:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,0:00,12:00,00:00,00:00,00:00,00:00,12:00


## DF Concatenation

In [ ]:
def df_concat(folder_path):
    import pandas as pd
    import os
    dfs = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            dfs.append(df)
    concatenated_df = pd.concat(dfs, ignore_index=True)

    #concatenated_df.to_csv(csv_name, index=False)

    return concatenated_df

In [ ]:
df_concat(#folder path, "goals.csv")
df_concat(#folder path "players.csv")
df_concat(#folder path, "minutesh1.csv")
df_concat(#folder path, "minutesh2.csv")
df_concat(#folder path, "match.csv")